In [118]:
import os, sys
import glob
import numpy as np

import bz2
import json
import re
import time

## parsing dumps
import mwxml
import mwparserfromhell as mwph
import glob
from collections import Counter

import time
import gzip
import fasttext
import fasttext.util
PATH_FT_MODEL = '/home/mgerlach/REPOS/topic-models/data/'

In [27]:
ft = fasttext.load_model(os.path.join(PATH_FT_MODEL,'cc.en.300.bin'))
## reduce model-size (if needed)
N_dim = 50
if N_dim <300:
    fasttext.util.reduce_model(ft, N_dim)

In [119]:
paths = []
wiki = 'simplewiki'
wiki = 'enwiki'

snapshot = '20200801'

dump_fn = '/mnt/data/xmldatadumps/public/{0}/{1}/{0}-{1}-pages-articles.xml.bz2'.format(wiki,snapshot)
for infile in glob.glob('/mnt/data/xmldatadumps/public/{0}/{1}/{0}-{1}-pages-articles*.xml*.bz2'.format(wiki,snapshot) ):
    if infile == dump_fn:
        continue
    if 'multistream' in infile:
        continue
    paths += [infile]
if len(paths) == 0:
    paths+=[dump_fn]
# paths = [dump_fn]
paths 

['/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles27.xml-p63663462p64710505.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles25.xml-p36952816p38067202.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles12.xml-p3926862p5040436.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles27.xml-p47163462p48663461.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles27.xml-p59163462p60663461.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles11.xml-p3046513p3926861.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles17.xml-p11539267p13039266.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles27.xml-p56163462p57663461.bz2',
 '/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles4.xml-p200510p352689.bz2',
 '/mnt/data/xmldata

In [120]:
def process_dump(dump, path):
    for page in dump:
        if page.redirect or page.namespace != 0:
            continue
        page_id = page.id
        page_title = page.title
        page_wikitext = mwph.parse(next(page).text)
        page_sections = page_wikitext.get_sections()
        page_text = page_sections[0].strip_code().replace('\n',' ')
        page_vec = ft.get_sentence_vector(page_text)
        
        yield page_id, page_title, page_text, page_vec
    print("Done processing path:", path)

In [121]:
N_kept = 0
t1 = time.time()
with gzip.open('../output/text/embedding-text_fasttext_%s_%s_%s.vec.gz'%(wiki,snapshot,N_dim),'wt') as fout:
    for page_id, page_title, page_text,page_vec in mwxml.map(process_dump, paths,threads=10):        
        str_out = '%s'%(page_id)
        for xi in page_vec:
            str_out += ' %.5f'%xi
        str_out+='\n'
        fout.write(str_out)
        N_kept+=1
t2 = time.time()
print(t2-t1)
print(N_kept)

Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles14.xml-p7697595p7744800.bz2
Done processing path: 
/mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles2.xml-p30304p88444.bz2Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles4.xml-p200510p352689.bz2
Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles27.xml-p63663462p64710505.bz2
Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles3.xml-p88445p200509.bz2
Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles20.xml-p20254736p21222156.bz2
Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles16.xml-p11018049p11539266.bz2
Done processing path: /mnt/data/xmldatadumps/public/enwiki/20200801/enwiki-20200801-pages-articles26.xml-p42567203p42663461.